In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer
import numpy as np
import seaborn as sns
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputRegressor 
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.svm import SVR

In [2]:

na_values=['',' ','Nan','NaN',-999.0,-999]
train=pd.read_csv('https://raw.githubusercontent.com/instabaines/PDDA/master/train.csv',na_values=na_values)
test=pd.read_csv('https://raw.githubusercontent.com/instabaines/PDDA/master/real_result_20perc.csv',na_values=na_values)
test2=pd.read_csv('https://raw.githubusercontent.com/instabaines/PDDA/master/test.csv',na_values=na_values)

In [3]:
train=train.dropna(axis=0)
test=test.dropna(axis=0)
test.shape

(2217, 9)

In [4]:

train['NGR']=0+(200-0)*(train['GR']-train['GR'].min())/(train['GR'].max()-train['GR'].min())
train['IGR']=(train['NGR']-train['NGR'].min())/(train['NGR'].max()-train['NGR'].min())
test['NGR']=0+(200-0)*(test['GR']-test['GR'].min())/(test['GR'].max()-test['GR'].min())
test['IGR']=(test['NGR']-test['NGR'].min())/(test['NGR'].max()-test['NGR'].min())
train['HRD']=np.log10(train['HRD'])
test['HRD']=np.log10(test['HRD'])

#del train['GR']
train=train.dropna(axis=0)
test=test.dropna(axis=0)


In [5]:
test2['NGR']=0+(200-0)*(test2['GR']-test2['GR'].min())/(test2['GR'].max()-test2['GR'].min())
test2['IGR']=(test2['NGR']-test2['NGR'].min())/(test2['NGR'].max()-test2['NGR'].min())

test2['HRD']=np.log10(test2['HRD'])

In [6]:
train

,CAL,CNC,GR,HRD,HRM,PE,ZDEN,DTC,DTS,NGR,IGR
573,14.8824,0.5173,57.1903,0.297235,1.9726,27.9432,2.3510,125.9611,306.8676,7.643731,0.038219
574,14.9570,0.4966,63.1593,0.311160,1.6373,28.1064,2.4055,125.9726,305.8347,8.456274,0.042281
575,14.6341,0.4433,64.5217,0.310991,1.4866,26.5968,2.4579,125.8528,305.9089,8.641733,0.043209
576,14.2318,0.4309,63.0144,0.302201,1.4613,19.5635,2.4219,125.7362,305.1737,8.436549,0.042183
577,14.0163,0.4469,60.8421,0.290769,1.4438,9.8556,2.3138,125.9774,304.4476,8.140840,0.040704
...,...,...,...,...,...,...,...,...,...,...,...
30138,8.6719,0.1879,119.9486,0.400192,2.5187,0.0597,2.6240,74.2133,131.7747,16.186840,0.080934
30139,8.6719,0.1825,122.5286,0.398131,2.4818,0.0582,2.6194,74.3503,132.2201,16.538048,0.082690
30140,8.6797,0.1774,122.0544,0.398114,2.4717,0.0571,2.6169,74.4752,132.5171,16.473497,0.082367
30141,8.6719,0.1744,120.6518,0.400538,2.5007,0.0562,2.6159,74.5917,132.7115,16.282565,0.081413


In [10]:
train.head()

,CAL,CNC,GR,HRD,HRM,PE,ZDEN,DTC,DTS,NGR,IGR
573,14.8824,0.5173,57.1903,0.297235,1.9726,27.9432,2.3510,125.9611,306.8676,7.643731,0.038219
574,14.9570,0.4966,63.1593,0.311160,1.6373,28.1064,2.4055,125.9726,305.8347,8.456274,0.042281
575,14.6341,0.4433,64.5217,0.310991,1.4866,26.5968,2.4579,125.8528,305.9089,8.641733,0.043209
576,14.2318,0.4309,63.0144,0.302201,1.4613,19.5635,2.4219,125.7362,305.1737,8.436549,0.042183
577,14.0163,0.4469,60.8421,0.290769,1.4438,9.8556,2.3138,125.9774,304.4476,8.140840,0.040704


In [8]:
test.head()

,CAL,CNC,GR,HRD,HRM,PE,ZDEN,DTC,DTS,NGR,IGR
0,8.6719,0.1199,6.7518,0.451556,3.0391,8.4395,2.5191,67.5481,124.2332,1.103719,0.005519
1,8.6719,0.3180,165.4345,0.119586,1.2206,7.1354,2.5517,101.5208,201.2649,32.851151,0.164256
2,8.7031,0.3164,58.3146,0.065468,1.8947,8.1722,2.4452,110.8297,250.7232,11.419818,0.057099
3,8.6250,0.1527,44.5126,0.403429,3.0052,7.6534,2.5674,80.0383,147.3262,8.658471,0.043292
4,8.6250,0.1731,16.1563,1.130597,13.0191,5.1534,2.2164,85.6109,134.4386,2.985264,0.014926


In [9]:
test2.head()

,CAL,CNC,HRD,HRM,PE,ZDEN,IGR,NGR
0,8.5781,0.3521,-0.090390,0.7810,6.8291,2.3256,0.048354,9.670859
1,8.5781,0.3639,-0.094852,0.7723,6.8100,2.3255,0.049982,9.996422
2,8.5781,0.3703,-0.128194,0.7048,6.7766,2.3212,0.051686,10.337276
3,8.5625,0.3667,-0.144541,0.6542,6.7219,2.3119,0.050266,10.053276
4,8.5781,0.3500,-0.164627,0.6109,6.6384,2.2982,0.046467,9.293497


In [7]:

X,y=train[['CAL', 'CNC', 'HRD', 'HRM', 'PE', 'ZDEN','IGR','NGR']],train[['DTC', 'DTS']]
test2=test2[X.columns]
X_train, X_test, y_train, y_test =X,test[X.columns],y,test[y.columns] 

#X['LI']=np.where(X['GR']>70,1,0)
#X=X.values
#y=y.values

In [ ]:
train

,CAL,CNC,GR,HRD,HRM,PE,ZDEN,DTC,DTS,IGR
573,14.8824,0.5173,1.757322,0.297235,0.295039,27.9432,2.3510,125.9611,306.8676,0.552475
574,14.9570,0.4966,1.800437,0.311160,0.214128,28.1064,2.4055,125.9726,305.8347,0.566159
575,14.6341,0.4433,1.809706,0.310991,0.172194,26.5968,2.4579,125.8528,305.9089,0.569100
576,14.2318,0.4309,1.799440,0.302201,0.164739,19.5635,2.4219,125.7362,305.1737,0.565842
577,14.0163,0.4469,1.784204,0.290769,0.159507,9.8556,2.3138,125.9774,304.4476,0.561007
...,...,...,...,...,...,...,...,...,...,...
30138,8.6719,0.1879,2.078995,0.400192,0.401176,0.0597,2.6240,74.2133,131.7747,0.654567
30139,8.6719,0.1825,2.088237,0.398131,0.394767,0.0582,2.6194,74.3503,132.2201,0.657500
30140,8.6797,0.1774,2.086553,0.398114,0.392996,0.0571,2.6169,74.4752,132.5171,0.656966
30141,8.6719,0.1744,2.081534,0.400538,0.398062,0.0562,2.6159,74.5917,132.7115,0.655373


In [ ]:
X_train.shape

(0, 7)

In [ ]:

#X_train, X_test, y_train, y_test =X,test[X.columns],y,test[y.columns] #train_test_split(X, y, random_state=0)
scaler=StandardScaler().fit(X_train)
X_train=scaler.transform(X_train)
X_test=scaler.transform(X_test)



In [ ]:
parameters = {'estimator__kernel': ('linear', 'rbf','poly'), 'estimator__C':[1.5, 10],'estimator__gamma': [1e-7, 1e-4],'estimator__epsilon':[0.1,0.2,0.5,0.3]}
svr =MultiOutputRegressor(SVR())
clf = RandomizedSearchCV(svr, parameters,n_jobs=-1,cv=5,scoring='neg_mean_squared_error')
clf.fit(Xscaled_train,y_train)
clf.best_params_

{'estimator__C': 10,
 'estimator__epsilon': 0.5,
 'estimator__gamma': 0.0001,
 'estimator__kernel': 'rbf'}

In [ ]:
predict=clf.predict(Xscaled_test)
rmse = np.sqrt(mean_squared_error(y_test, predict))
print("MAE: %f" % (rmse))

MAE: 28.410401


In [ ]:
predict=xgb.predict(X_test)

In [ ]:
rmse = np.sqrt(mean_squared_error(y_test, predict))
print("RMSE: %f" % (rmse))
print('\n')
#print("Classification Report", classification_report(y_test,predict))
print('\n')
#print("Confusion Matrics", confusion_matrix(y_test,predict))

RMSE: 85.671998






In [ ]:
param_test1={'estimator__max_depth':range(1,10),'estimator__min_child_weight': range(1,6),'estimator__learning_rate':[i/10.0 for i in range(0,6)],'estimator__n_estimators':(80,100,120,140,),'estimator__gamma':[i/10.0 for i in range(0,5)],'estimator__subsample':(0.8,0.9,0.7),'estimator__colsample_bytree':(0.8,0.7), 'estimator__objective':('reg:linear','reg:squarederror',),'estimator__nthread':(4,), 'estimator__scale_pos_weight':(1,),'estimator__seed':(27,)}


In [ ]:
xg_reg = (XGBRegressor())
gsearch1=RandomizedSearchCV(estimator=MultiOutputRegressor(xg_reg),scoring='neg_mean_squared_error',param_distributions=param_test1,n_jobs=-1,iid=False, cv=5,random_state=123456)
gsearch1.fit(X_train,y_train)
gsearch1.cv_results_,gsearch1.best_params_,gsearch1.best_score_

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_search.py:823: FutureWarning: The parameter 'iid' is deprecated in 0.22 and will be removed in 0.24.
  "removed in 0.24.", FutureWarning


({'mean_fit_time': array([9.05638857, 7.54878416, 6.7941361 , 3.12290277, 5.45981317,
         2.21655922, 7.54049287, 2.69735246, 8.16378884, 2.99843392]),
  'mean_score_time': array([0.08119688, 0.0662303 , 0.06714096, 0.03450933, 0.05479326,
         0.02279038, 0.07238026, 0.02829862, 0.08859921, 0.0287519 ]),
  'mean_test_score': array([-2251.0601685 , -2143.00529793, -2332.10896043, -2058.603404  ,
         -2174.4671976 , -2166.24325826, -2307.57294925, -1839.21573385,
         -2212.87677272, -2330.99811634]),
  'param_estimator__colsample_bytree': masked_array(data=[0.8, 0.8, 0.7, 0.7, 0.7, 0.7, 0.8, 0.8, 0.7, 0.7],
               mask=[False, False, False, False, False, False, False, False,
                     False, False],
         fill_value='?',
              dtype=object),
  'param_estimator__gamma': masked_array(data=[0.0, 0.3, 0.3, 0.0, 0.0, 0.1, 0.2, 0.3, 0.1, 0.3],
               mask=[False, False, False, False, False, False, False, False,
                     Fals

In [ ]:
best_params={'estimator__colsample_bytree': 0.8,
  'estimator__gamma': 0.0,
  'estimator__learning_rate': 0.1,
  'estimator__max_depth': 9,
  'estimator__min_child_weight': 1,
  'estimator__n_estimators': 140,
  'estimator__nthread': 4,
  'estimator__objective': 'reg:linear',
  'estimator__scale_pos_weight': 1,
  'estimator__seed': 27,
  'estimator__subsample': 0.8}

In [ ]:
xgb=MultiOutputRegressor(XGBRegressor(colsample_bytree= 0.8,
gamma= 0.1,
  learning_rate= 0.1,
  max_depth= 8,
  min_child_weight= 2,
  n_estimators= 100,
  nthread= 4,
  objective= 'reg:squarederror',
  scale_pos_weight= 1,
  seed= 27,
  subsample= 0.8, random_state=42)).fit(X_train,y_train)

In [ ]:
predict=xgb.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, predict))
print("MAE: %f" % (rmse))
print('\n')
#print("Classification Report", classification_report(y_test,predict))
print('\n')
#print("Confusion Matrics", confusion_matrix(y_test,predict))


MAE: 17.144052






In [ ]:
#17.144052
final_predict=xgb.predict(test2)
result=pd.DataFrame({'DTC': [x[0] for x in final_predict],'DTS':[x[1] for x in final_predict]})

In [ ]:
result

,DTC,DTS
0,112.448494,259.722626
1,112.448494,259.722626
2,112.802727,258.453583
3,112.448494,258.453583
4,112.448494,256.451019
...,...,...
11083,113.030556,257.434204
11084,113.030556,257.434204
11085,113.815422,257.434204
11086,113.815422,257.434204


In [ ]:
from google.colab import drive
drive.mount('drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at drive


In [ ]:
result.to_csv('result.csv')
!cp result.csv "drive/My Drive/"

In [ ]:
result.shape

(11088, 2)